In [ ]:
%pip install qiskit

In [ ]:
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.circuit.library import MCXGate

def int_to_bits(x, n):
    """Convert integer x to list of n bits (little endian)."""
    return [(x >> i) & 1 for i in range(n)]

def controlled_f(qc, x_qubits, f, f_out_qubits, ancillas=[]):
    """Evaluate pseudo-boolean function f(x) into output register f_out_qubits."""
    # For demonstration: assume f is a sum f(x) = sum(c_i * x_i), with c_i in {1,2,...}
    # This makes quantum circuit design tractable.
    # User can extend this as needed for more complex f.
    n = len(x_qubits)
    for idx, coeff in enumerate(f['coeffs']):    # f given as {'coeffs': [...], 'offset': ...}
        for j in range(coeff):
            qc.cx(x_qubits[idx], f_out_qubits[j]) # naive implementation, 1 output qubit per increment
    # Offset is classical, can add with X gates
    for j in range(f['offset']):
        qc.x(f_out_qubits[j])

def controlled_C(qc, x_qubits, cgate, C_out_qubit, ancillas=[]):
    """Evaluate constraint function C(x) and output 1 if C(x)==0."""
    # For demonstration: assume C(x) = sum(d_i * x_i) == 0 mod 2 (parity constraint)
    # Extendible for other C(x)
    n = len(x_qubits)
    for idx, coeff in enumerate(cgate['coeffs']):
        for j in range(abs(coeff)):
            if coeff > 0:
                qc.cx(x_qubits[idx], C_out_qubit)
            else:
                qc.x(x_qubits[idx])
                qc.cx(x_qubits[idx], C_out_qubit)
                qc.x(x_qubits[idx])

def marker_oracle(n, f, t, C):
    """
    Returns a QuantumCircuit for the Grover marker oracle:
      U_{f,t,C}(|x>|y>) = |x>|y ⊕ 1>  if f(x) ≥ t and C(x) = 0,
                            |x>|y>    otherwise
    Where f and C are specified in dict form (see above).
    """
    x_qubits = QuantumRegister(n, 'x')
    y_qubit = QuantumRegister(1, 'y')
    f_qubits = QuantumRegister(n, 'freg')       # For f(x) output (expand as needed)
    C_qubit = QuantumRegister(1, 'c')           # For constraint check
    qc = QuantumCircuit(x_qubits, y_qubit, f_qubits, C_qubit)

    # Step 1: Compute f(x) into f_qubits
    controlled_f(qc, x_qubits, f, f_qubits)

    # Step 2: Compute C(x) == 0 into C_qubit
    controlled_C(qc, x_qubits, C, C_qubit)

    # Step 3: Check condition f(x) >= t and C(x)==0 and flip y_qubit if true
    # Compare f(x) >= t (for demo: t is 1, so just check if any f_qubits are 1)
    flip_controls = []
    for j in range(t, n):   # adjust for t
        flip_controls.append(f_qubits[j])
    flip_controls.append(C_qubit[0])  # constraint must be 0 triggers flip

    qc.mcx(flip_controls, y_qubit[0])

    # Step 4: Uncompute f(x) and C(x)
    controlled_C(qc, x_qubits, C, C_qubit)      # Uncompute
    controlled_f(qc, x_qubits, f, f_qubits)     # Uncompute

    return qc

# EXAMPLE USAGE FOR n=2
f = {'coeffs': [2, 1], 'offset': 0}    # f(x) = 2*x0 + 1*x1
C = {'coeffs': [1, -1]}                # C(x) = x0 - x1
t = 2                                  # Set threshold

oracle = marker_oracle(2, f, t, C)
print(oracle.draw())


After installing the library, please run the code cell again.